In [1]:
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras as ks

In [2]:
dataset = pd.read_csv('sensor.csv')
dataset.head()

,Node_ID,Timestamp,IP_Address,Packet_Rate,Packet_Drop_Rate,Packet_Duplication_Rate,Data_Throughput,Signal_Strength,SNR,Battery_Level,...,Number_of_Neighbors,Route_Request_Frequency,Route_Reply_Frequency,Data_Transmission_Frequency,Data_Reception_Frequency,Error_Rate,CPU_Usage,Memory_Usage,Bandwidth,Is_Malicious
0,1,1/2/2023 0:00,192.168.119.138,52.018229,2.727317,0.955687,130.967180,-52.059232,23.974265,82.982598,...,3,11.323044,11.458199,20.670539,14.385439,0.109146,26.376693,29.186341,76.811986,0
1,2,1/2/2023 0:01,192.168.225.56,59.504648,1.435058,1.228460,82.634358,-57.754723,22.572052,66.734853,...,6,8.277362,8.832980,15.555122,14.316681,0.077136,34.963429,34.203177,112.495912,0
2,3,1/2/2023 0:02,192.168.187.210,34.118204,1.320905,1.015245,61.649928,-52.399037,15.667076,74.593605,...,5,8.196374,8.330227,19.470748,11.792084,0.052993,48.934585,31.682662,117.526109,0
3,4,1/2/2023 0:03,192.168.78.245,39.643055,1.988791,0.784562,69.704181,-53.727443,20.286962,76.710564,...,6,10.683766,6.525032,16.228955,8.895409,0.087810,47.045600,37.470027,123.058913,0
4,5,1/2/2023 0:04,192.168.233.229,40.212679,1.005121,1.412193,123.445612,-49.696620,24.318538,84.477890,...,7,11.176823,10.617432,12.257423,10.168739,0.068823,33.394354,26.582970,71.696911,0


In [3]:
# Find NAN
nan_df = dataset[dataset.isna().any(axis=1)]
display(nan_df.head())
dataset = dataset.dropna(how='all')

,Node_ID,Timestamp,IP_Address,Packet_Rate,Packet_Drop_Rate,Packet_Duplication_Rate,Data_Throughput,Signal_Strength,SNR,Battery_Level,...,Number_of_Neighbors,Route_Request_Frequency,Route_Reply_Frequency,Data_Transmission_Frequency,Data_Reception_Frequency,Error_Rate,CPU_Usage,Memory_Usage,Bandwidth,Is_Malicious


In [4]:
counts = dataset['Is_Malicious'].value_counts()
counts_filtered = counts.loc[[0, 1]]
print(counts_filtered)

Is_Malicious
0    9513
1     487
Name: count, dtype: int64


In [5]:
dataset.shape

(10000, 21)

In [6]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Assuming 'dataset' is your DataFrame containing the whole dataset
enc = LabelEncoder()

# Convert 'Timestamp' and 'IP_Address' columns to numeric representation
dataset[[ 'IP_Address']] = \
    dataset[['IP_Address']].apply(enc.fit_transform)

# Convert 'Timestamp' column to UNIX timestamp (seconds)
dataset['Timestamp'] = pd.to_datetime(dataset['Timestamp']).astype('int64') // 10**9

# Display the modified DataFrame
print(dataset.head())


   Node_ID   Timestamp  IP_Address  Packet_Rate  Packet_Drop_Rate  \
0        1  1672617600         861    52.018229          2.727317   
1        2  1672617660        5081    59.504648          1.435058   
2        3  1672617720        3510    34.118204          1.320905   
3        4  1672617780        8345    39.643055          1.988791   
4        5  1672617840        5429    40.212679          1.005121   

   Packet_Duplication_Rate  Data_Throughput  Signal_Strength        SNR  \
0                 0.955687       130.967180       -52.059232  23.974265   
1                 1.228460        82.634358       -57.754723  22.572052   
2                 1.015245        61.649928       -52.399037  15.667076   
3                 0.784562        69.704181       -53.727443  20.286962   
4                 1.412193       123.445612       -49.696620  24.318538   

   Battery_Level  ...  Number_of_Neighbors  Route_Request_Frequency  \
0      82.982598  ...                    3                11.32

In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Assuming dataset is a pandas DataFrame
scaler = MinMaxScaler()
dataset_scaled = scaler.fit_transform(dataset)

# Convert the scaled NumPy array back to a DataFrame with original column names
dataset_scaled_df = pd.DataFrame(dataset_scaled, columns=dataset.columns)
dataset_scaled_df.head()



,Node_ID,Timestamp,IP_Address,Packet_Rate,Packet_Drop_Rate,Packet_Duplication_Rate,Data_Throughput,Signal_Strength,SNR,Battery_Level,...,Number_of_Neighbors,Route_Request_Frequency,Route_Reply_Frequency,Data_Transmission_Frequency,Data_Reception_Frequency,Error_Rate,CPU_Usage,Memory_Usage,Bandwidth,Is_Malicious
0,0.0000,0.000000,0.093202,0.370660,0.421887,0.483303,0.726891,0.387964,0.654562,0.588587,...,0.357143,0.438084,0.364206,0.538487,0.360808,0.258052,0.296612,0.399257,0.362036,0.0
1,0.0001,0.000004,0.550011,0.446889,0.240218,0.660597,0.432764,0.261806,0.621356,0.403047,...,0.571429,0.290589,0.252353,0.376536,0.358555,0.157552,0.390018,0.505119,0.606860,0.0
2,0.0002,0.000008,0.379952,0.188397,0.224170,0.522014,0.305064,0.380438,0.457839,0.492789,...,0.500000,0.286667,0.230932,0.500502,0.275840,0.081752,0.541996,0.451933,0.641372,0.0
3,0.0003,0.000011,0.903334,0.244653,0.318063,0.372078,0.354078,0.351013,0.567243,0.516964,...,0.571429,0.407125,0.154018,0.397869,0.180935,0.191066,0.521447,0.574054,0.679332,0.0
4,0.0004,0.000015,0.587681,0.250453,0.179776,0.780017,0.681119,0.440298,0.662715,0.605663,...,0.642857,0.431003,0.328383,0.272133,0.222654,0.131454,0.372950,0.344322,0.326941,0.0


In [8]:
from imblearn.over_sampling import SMOTE
X = dataset_scaled_df.drop(columns=['Is_Malicious'])  # Features
y = dataset_scaled_df['Is_Malicious']  # Target

# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Now, X_resampled and y_resampled contain the resampled data

c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File

In [9]:
from collections import Counter
print("Before SMOTE :" , Counter(y))
print("After SMOTE :" , Counter(y_resampled))

Before SMOTE : Counter({0.0: 9513, 1.0: 487})
After SMOTE : Counter({0.0: 9513, 1.0: 9513})


In [10]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Split the original dataset into training and testing datasets
X_train, X_val_test, y_train, y_val_test = train_test_split(X_resampled, y_resampled, test_size=0.3)
X_test, X_val, y_test, y_val = train_test_split(X_val_test, y_val_test, test_size=0.5)

In [11]:
# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything other feature

def correlation(dataset_scaled_df, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset_scaled_df.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [12]:
corr_features = correlation(X_train, 0.65)
len(set(corr_features))

5

In [13]:
corr_features

{'Data_Throughput',
 'Data_Transmission_Frequency',
 'Energy_Consumption_Rate',
 'Error_Rate',
 'Timestamp'}

In [14]:
# Dropping
columns_to_drop = ['Data_Throughput',
 'Data_Transmission_Frequency',
 'Energy_Consumption_Rate',
 'Error_Rate',
 'Timestamp'] 
dataset_scaled_df= dataset_scaled_df.drop(columns=columns_to_drop)  # Features


In [15]:
import pandas as pd

# Convert X_val_smote to a pandas DataFrame
X_val= pd.DataFrame(X_val)
X_test = pd.DataFrame(X_test)
# Drop correlated features from X_val_smote
X_val = X_val.drop(corr_features, axis=1)
X_test = X_test.drop(corr_features, axis=1)
# Convert the modified DataFrame back to a NumPy array if needed
X_val = X_val.to_numpy()
X_test = X_test.to_numpy()
X_train = X_train.drop(corr_features, axis=1)
X_train = X_train.to_numpy()

In [19]:
#Random Forest Model
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor()
forest.fit(X_train,y_train)

RandomForestRegressor()

In [20]:
forest.score(X_val,y_val)

0.9939021299734748

In [22]:
input_data = (0.0,0.09320199177311106,0.37066024378577744,0.4218865165416059,0.48330341091121426,0.38796441189988196,0.6545619840487272,0.5885871233858044,0.3571428571428571,0.4380838880347805,0.3642055757305056,0.36080763325062676,0.2966116226538069,0.3992566419091834,0.36203561853680466)

# changing input data to a numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the numpy array
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = forest.predict(input_data_reshaped)
print(prediction)


if (prediction[0] == 0):
  print("This node is Not Malicious!")

else:
  print("This node is Malicious")

[0.]
This node is Not Malicious!


In [23]:
import pickle


pickle.dump(forest,open('rf_model.pkl','wb'))